In [3]:
import sys
sys.path.append("../../pygenesig")

%load_ext autoreload
%autoreload 1
%aimport pygenesig
%matplotlib inline

import numpy as np
import pandas as pd
from collections import OrderedDict
import dask.dataframe as dd


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Preprocess FANTOM5 data

In this notebook, we process the fantom5 data for the use with *pygenesig* so that we can easily create and validate signatures. 

For the crossvalidation we need a
* matrix of gene expression data
* list of target classes (signatures) 

Here, we extract the relevant information from the fantom5 expression and annotation files

In [5]:
!ls ../data

annotation_notes.csv	     fantom5-S1.xls
biolayout		     ff-phase2-140729.corr.obo
column_vars.processed.csv    ff-phase2-140729.obo
column_vars.txt		     hg19.cage_peak_phase1and2combined_ann.txt
corr_mat.primary.tsv	     hg19.cage_peak_phase1and2combined_tpm_ann.osc.txt
delimiter_nodes.tsv	     missing_samples.txt
f5_eset_primary_cells.Rdata  process_sample_descriptions.log
f5_expressionset.Rdata	     pygenesig
fantom5_head2000.txt	     tmp


In [6]:
col_vars = pd.read_csv("../data/column_vars.processed.csv", sep=",", index_col=0)

In [7]:
gct = pd.read_csv("../data/hg19.cage_peak_phase1and2combined_tpm_ann.osc.txt", sep="\t", 
                  index_col=0, comment='#')
gct = gct.iloc[4:, ] # remove stats

In [8]:
gene_id_map = pd.read_csv("../data/pygenesig/FANTOM5cage_TX_ELEMENTS", sep="\t")
f5_to_geneid = dict(zip(gene_id_map.PROMOTERID, gene_id_map.GENEID))

In [9]:
gct = gct[gct.short_description.isin(f5_to_geneid.keys())] # gene symbols only 

In [10]:
geneids = gct.short_description.apply(lambda x: f5_to_geneid[x])

## tissues
Extract data for tissue samples

In [11]:
col_vars_t = col_vars[col_vars.sample_type == "tissue"]
gct_t = gct[col_vars_t.index]

In [13]:
col_vars_t["tissue"] = col_vars_t.name.apply(lambda x: x.split(",")[0].lower())

/pstore/apps/Anaconda3/2.5.0/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [14]:
grouped = col_vars_t.groupby("tissue")
tissue_count = grouped.count().name
tissue_count

tissue
achilles tendon                                  1
adipose                                          4
adipose tissue                                   1
adrenal gland                                    1
amygdala                                         2
aorta                                            1
appendix                                         1
artery                                           1
bladder                                          1
blood                                            1
bone marrow                                      1
brain                                            3
breast                                           1
caudate nucleus                                  3
cerebellum                                       2
cerebellum - adult                               1
cerebral meninges                                1
cerebrospinal fluid                              1
cervix                                           1
clontech human universal

### store results

In [16]:
exprs = gct_t.as_matrix()
np.save("../data/pygenesig/tissue_exprs.npy", exprs)

In [18]:
target = col_vars_t.tissue
np.savetxt("../data/pygenesig/tissue_target.csv", target, delimiter=",", fmt="%s")

In [19]:
geneids = geneids[gct_t.index]
geneids.to_csv("../data/pygenesig/tissue_gene_ids.csv")

## primary cells
Extract data for primary cell samples. 

We distinguish between *fine* and *coarse*. We take the pragmatic approach to rely on the sample name as annotation. 
We split the sample name at the dash. The part before the dash is 'coarse' the part behind 'fine'. 

For the *coarse* dataset, we discard all cell types that do not have at least 2 samples. 

In [20]:
col_vars_p = col_vars[col_vars.sample_type == "primary cell"]
gct_p = gct[col_vars_p.index]

In [21]:
col_vars_p["type_fine"] = col_vars_p.name.apply(lambda x: x.split(",")[0].lower())
col_vars_p["type_coarse"] = col_vars_p.type_fine.apply(lambda x: x.split("-")[0].strip().lower())

/pstore/apps/Anaconda3/2.5.0/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/pstore/apps/Anaconda3/2.5.0/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [22]:
grouped = col_vars_p.groupby("type_fine")
tissue_count = grouped.count().name
tissue_count

type_fine
adipocyte - breast                                                      2
adipocyte - omental                                                     3
adipocyte - perirenal                                                   1
adipocyte - subcutaneous                                                3
alveolar epithelial cells                                               2
amniotic epithelial cells                                               3
amniotic membrane cells                                                 3
anulus pulposus cell                                                    2
aortic smooth muscle cell response to fgf2                              2
aortic smooth muscle cell response to il1b                              2
astrocyte - cerebellum                                                  3
astrocyte - cerebral cortex                                             3
basophils                                                               3
bronchial epithelial cell   

In [38]:
grouped = col_vars_p.groupby("type_coarse")
with pd.option_context('display.max_rows', None):
    tissue_count = grouped.count().name
    print(tissue_count)

type_coarse
adipocyte                                                   9
alveolar epithelial cells                                   2
amniotic epithelial cells                                   3
amniotic membrane cells                                     3
anulus pulposus cell                                        2
aortic smooth muscle cell response to fgf2                  2
aortic smooth muscle cell response to il1b                  2
astrocyte                                                   6
basophils                                                   3
bronchial epithelial cell                                   7
cardiac myocyte                                             3
cd133                                                       2
cd14                                                       42
cd14+ monocyte derived endothelial progenitor cells         3
cd19                                                       11
cd34                                                      

Choos only cells that have at least 2 samples (otherwise we can't do crossvalidation and are even more likely to learn batch effects) 

In [46]:
multi_sample_tissues = list(tissue_count.index[tissue_count >= 2])

In [48]:
col_vars_p = col_vars_p[col_vars_p.type_coarse.isin(multi_sample_tissues)]
gct_p = gct[col_vars_p.index]

In [49]:
geneids = geneids[gct_p.index]

### store data

In [50]:
exprs = gct_p.as_matrix()
np.save("../data/pygenesig/primary_exprs.npy", exprs)

In [51]:
np.savetxt("../data/pygenesig/primary_target_fine.csv", col_vars_p.type_fine, delimiter=",", fmt="%s")
np.savetxt("../data/pygenesig/primary_target_coarse.csv", col_vars_p.type_coarse, delimiter=",", fmt="%s")

In [64]:
col_vars_p.to_csv("../data/pygenesig/primary_col_vars.csv")

In [52]:
geneids = geneids[gct_p.index]
geneids.to_csv("../data/pygenesig/primary_gene_ids.csv")

In [63]:
col_vars_p

,biol_rep,donor,lib_id,name,name_orig,obo_id,sample_type,tech_rep,time,type_fine,type_coarse
74,True,donor1,CNhs11051,"Adipocyte - breast, donor1","tpm of Adipocyte - breast, donor1.CNhs11051.11...",FF:11376-118A8,primary cell,False,NaN,adipocyte - breast,adipocyte
75,True,donor2,CNhs11969,"Adipocyte - breast, donor2","tpm of Adipocyte - breast, donor2.CNhs11969.11...",FF:11327-117E4,primary cell,False,NaN,adipocyte - breast,adipocyte
76,True,donor1,CNhs11054,"Adipocyte - omental, donor1","tpm of Adipocyte - omental, donor1.CNhs11054.1...",FF:11473-119C6,primary cell,False,NaN,adipocyte - omental,adipocyte
77,True,donor2,CNhs12067,"Adipocyte - omental, donor2","tpm of Adipocyte - omental, donor2.CNhs12067.1...",FF:11474-119C7,primary cell,False,NaN,adipocyte - omental,adipocyte
78,True,donor3,CNhs12068,"Adipocyte - omental, donor3","tpm of Adipocyte - omental, donor3.CNhs12068.1...",FF:11475-119C8,primary cell,False,NaN,adipocyte - omental,adipocyte
79,True,donor1,CNhs12069,"Adipocyte - perirenal, donor1","tpm of Adipocyte - perirenal, donor1.CNhs12069...",FF:11476-119C9,primary cell,False,NaN,adipocyte - perirenal,adipocyte
80,True,donor1,CNhs12494,"Adipocyte - subcutaneous, donor1","tpm of Adipocyte - subcutaneous, donor1.CNhs12...",FF:11259-116F8,primary cell,False,NaN,adipocyte - subcutaneous,adipocyte
81,True,donor2,CNhs11371,"Adipocyte - subcutaneous, donor2","tpm of Adipocyte - subcutaneous, donor2.CNhs11...",FF:11336-117F4,primary cell,False,NaN,adipocyte - subcutaneous,adipocyte
82,True,donor3,CNhs12017,"Adipocyte - subcutaneous, donor3","tpm of Adipocyte - subcutaneous, donor3.CNhs12...",FF:11408-118E4,primary cell,False,NaN,adipocyte - subcutaneous,adipocyte
95,True,donor1,CNhs11325,"Alveolar Epithelial Cells, donor1","tpm of Alveolar Epithelial Cells, donor1.CNhs1...",FF:11510-119G7,primary cell,False,NaN,alveolar epithelial cells,alveolar epithelial cells
